In [88]:
import os
from openai import OpenAI
from dotenv import load_dotenv
import json
import sqlite3
from typing import List, Dict, Any
import pandas as pd

#textwrap
import textwrap
from IPython.display import display
from IPython.display import Markdown

In [75]:
# Load environment variables from .env file
load_dotenv()

# Get the API key from environment variable
api_key = os.getenv("OPEN_AI_KEY")

# Create OpenAI client
client_openai = OpenAI(api_key=api_key)

# Define model
model = "gpt-4o-mini"

In [6]:
def get_analysis(prompt, model=model, max_tokens=1000, temperature=0.1):
  completion = client_openai.chat.completions.create(
    model=model,
    max_tokens=max_tokens,
    temperature=temperature,
    messages=[
      {"role": "user", "content": f"{prompt}"}
    ]
  )
  return completion.choices[0].message.content

In [36]:
def json_to_sqlite_memory_table(data: List[Dict[str, Any]], table_name: str = "data") -> sqlite3.Connection:
    """
    Creates an in-memory SQLite table from a JSON object (list of dictionaries).

    Args:
        data (List[Dict[str, Any]]): JSON data loaded into a Python list of dictionaries.
        table_name (str): Name of the SQLite table to be created.

    Returns:
        sqlite3.Connection: SQLite connection object with the table created in memory.
    """
    if not isinstance(data, list) or not all(isinstance(item, dict) for item in data):
        raise ValueError("Input data must be a list of dictionaries.")

    # Infer schema from the first item
    sample = data[0]
    column_definitions = []
    for key, value in sample.items():
        if isinstance(value, int):
            column_type = "INTEGER"
        elif isinstance(value, float):
            column_type = "REAL"
        else:
            column_type = "TEXT"
        column_definitions.append(f'"{key}" {column_type}')

    # Create in-memory SQLite database
    conn = sqlite3.connect(":memory:")
    cursor = conn.cursor()

    # Create the table
    columns_sql = ", ".join(column_definitions)
    sql_command = f'CREATE TABLE "{table_name}" ({columns_sql})'
    cursor.execute(sql_command)
    print(sql_command)
   
    # Prepare insert statement
    column_names = list(sample.keys())
    placeholders = ", ".join("?" for _ in column_names)
    insert_sql = f'INSERT INTO "{table_name}" ({", ".join(column_names)}) VALUES ({placeholders})'

    # Insert data
    for record in data:
        values = tuple(record.get(col) for col in column_names)
        cursor.execute(insert_sql, values)

    conn.commit()
    print(f"Table '{table_name}' created in memory with {len(data)} rows.")
    return conn


In [97]:
def run_and_print_query(conn: sqlite3.Connection, query: str):
    """
    Executes a SQL query on a given SQLite connection and prints the results with column names.

    Args:
        conn (sqlite3.Connection): Active SQLite connection.
        query (str): SQL query to execute.
    """
    cursor = conn.cursor()
    cursor.execute(query)

    # Get column names
    column_names = [description[0] for description in cursor.description]

    # Fetch all rows
    rows = cursor.fetchall()

    # Print column headers
    print(" | ".join(column_names))
    print("-" * (len(column_names) * 15))

    # Print rows
    for row in rows:
        print(" | ".join(str(item) if item is not None else "NULL" for item in row))


In [ ]:
def run_query(conn: sqlite3.Connection, query: str):
    """
    Executes a SQL query on a given SQLite connection and prints the results with column names.

    Args:
        conn (sqlite3.Connection): Active SQLite connection.
        query (str): SQL query to execute.
    """
   pd.read_sql(query, conn)
   #return df

In [100]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [37]:
# Table name
table_name = 'events'

# Load the JSON externally
with open("input/events.json", "r", encoding="utf-8") as f:
    json_data = json.load(f)

# Pass the JSON to the function
conn = json_to_sqlite_memory_table(json_data, table_name=table_name)

CREATE TABLE "events" ("event_guid" TEXT, "date" TEXT, "type" TEXT, "description" TEXT, "crm_id" INTEGER, "email" TEXT, "ip_address" TEXT, "user_agent" TEXT, "bot" INTEGER, "mailshot_id" INTEGER, "condition_id" INTEGER, "condition_title" TEXT, "condition_subject" TEXT, "condition_from" TEXT, "template_id" TEXT, "template_title" TEXT, "template_link" TEXT)
Table 'events' created in memory with 100 rows.


In [186]:
# Create a cursor
cursor = conn.cursor()

# Example query: select people over 40
#query = "SELECT * FROM events"
#cursor.execute(query)

# Fetch column names
#column_names = [description[0] for description in cursor.description]

# Fetch all results
#results = cursor.fetchall()

# Display with column names
#print(" | ".join(column_names))
#print("-" * 50)
#for row in results:
#    print(" | ".join(str(value) for value in row))

query = '''
    SELECT events.email, type, 
           LAG(type) OVER (PARTITION BY events.email ORDER BY date) AS previous_type
    FROM events, 
         (select email, count(*) as total from events group by email having count(*) > 1 ) as res
    WHERE events.email = res.email
'''

query = '''
   select * from (
   SELECT email AS user_email, type AS current_event_type, 
     LAG(type) OVER (PARTITION BY email ORDER BY date) AS previous_event_type 
     FROM events) as res
   where previous_event_type <> 'None' and current_event_type = 'emopened'
'''
cursor.execute(query)

# Fetch all results
results = cursor.fetchall()

# Display with column names
print(" | ".join(column_names))
print("-" * 50)
for row in results:
    print(" | ".join(str(value) for value in row))

event_guid | date | type | description | crm_id | email | ip_address | user_agent | bot | mailshot_id | condition_id | condition_title | condition_subject | condition_from | template_id | template_title | template_link
--------------------------------------------------
2116test@kulea.ma | emopened | emdelivery
28test@kulea.ma | emopened | emdelivery
3111test@kulea.ma | emopened | emdelivery
3237test@kulea.ma | emopened | emdelivery
7676test@kulea.ma | emopened | emdelivery
7676test@kulea.ma | emopened | emclickedlink
andrew@kulea.ma | emopened | emdelivery
andrew@kulea.ma | emopened | emdelivery
andrew@kulea.ma | emopened | emclickedlink
constantin@kulea.ma | emopened | emdelivery
constantin@kulea.ma | emopened | emdelivery
constantin@kulea.ma | emopened | emopened
constantin@kulea.ma | emopened | emdelivery
constantin@kulea.ma | emopened | emclickedlink
steve.bircher@btinternet.com | emopened | emdelivery
steve.bircher@btinternet.com | emopened | emopened
steve@kulea.ma | emopened | e

In [19]:
def get_table_schema(conn: sqlite3.Connection, table_name: str = "data"):
    cursor = conn.cursor()
    cursor.execute(f"PRAGMA table_info('{table_name}')")
    schema_info = cursor.fetchall()

    print(f"Schema for table '{table_name}':")
    for col in schema_info:
        col_id, name, col_type, notnull, default_value, pk = col
        print(f"- {name} ({col_type})")


In [ ]:
schema1 = get_table_schema(conn, table_name=table_name)
schema1

schema = 'CREATE TABLE "events" ("event_guid" TEXT, "date" TEXT, "type" TEXT, "description" TEXT, "crm_id" INTEGER, "email" TEXT, "ip_address" TEXT, "user_agent" TEXT, "bot" INTEGER, "mailshot_id" INTEGER, "condition_id" INTEGER, "condition_title" TEXT, "condition_subject" TEXT, "condition_from" TEXT, "template_id" TEXT, "template_title" TEXT, "template_link" TEXT)'
schema

schema = '''
[
  { "name": "event_guid", "type": "str", "example_values": null },
  { "name": "date", "type": "str", "example_values": null },
  { "name": "type", "type": "str", "example_values": ["emopened", "emclickedlink", "emdelivery"] },
  { "name": "description", "type": "str", "example_values": null },
  { "name": "crm_id", "type": "int", "example_values": null },
  { "name": "email", "type": "str", "example_values": null },
  { "name": "ip_address", "type": "str", "example_values": null },
  { "name": "user_agent", "type": "str", "example_values": null },
  { "name": "bot", "type": "bool", "example_values": [false, true] },
  { "name": "mailshot_id", "type": "int", "example_values": null },
  { "name": "condition_id", "type": "int", "example_values": null },
  { "name": "condition_title", "type": "str", "example_values": [
    "Example 1", "Form Thank you - Kulea demo1", "Test2", "test run",
    "Andrew Nicholson", "ccc", "eee", "Euro stagger unsubscribe test", "list-unsubscribe true"
  ]},
  { "name": "condition_subject", "type": "str", "example_values": [
    "Please don't open yet", "Wild test with to data", "Mel milking it", 
    "Marketing automation that cuts through the noise"
  ]},
  { "name": "condition_from", "type": "str", "example_values": [
    "andy@kulea.ma", "andrew@kulea.co.uk", "andrew@kulea.ma", 
    "roc@kulea.ma", "esther@kulea.ma"
  ]},
  { "name": "template_id", "type": "str", "example_values": [
    "fa7cf62d-963e-41c2-abd1-2980a86e3ec4", 
    "e49af120-9a1f-4ab2-8a15-db58cad776b5", 
    "e4cf3f50-dc87-4bc0-9e15-2aa6c2cb5a7e"
  ]},
  { "name": "template_title", "type": "str", "example_values": [
    "Test 1", "Wild.London", "Personal email", "Kulea branded email - Andrew"
  ]},
  { "name": "template_link", "type": "str", "example_values": [
    "Test-1", "Wild-London", "Personal-email", "Kulea-branded-email-Andrew"
  ]}
]
'''

In [175]:
query1 = 'How many email open events occurred?'
query2 = 'List all email addresses that clicked on a link.'
query3 = 'How many unique email addresses opened the email.?'
query4 = 'Count how many times each type of event occurred.'
query5 = 'Which email addresses generated more than one event?'
query6 = 'What are the most relevant condiction ids?'
query7 = 'What are the most relevant condiction ids and mailshot ids combined?'
query8 = '''List all emails that opened an event and the previous event type they interacted with. 
Use the LAG function. Also use the clausure WITH for the main query and after that filter 
the previous events column with different value from 'None'.'''

query = query8
query

"List all emails that opened an event and the previous event type they interacted with. \nUse the LAG function. Also use the clausure WITH for the main query and after that filter \nthe previous events column with different value from 'None'."

In [176]:
prompt = f'''Considering the database schema {schema} and the table name {table_name}, 
write an SQL query to answer the query: "{query}". The generated queries must attribute an alias for 
each column when is not used the column name. 
In the answer, present only the SQL query without any formatting or line breaks as a string, 
without the ";" character at the end and without the "\" character'''
prompt

'Considering the database schema \n[\n  { "name": "event_guid", "type": "str", "example_values": null },\n  { "name": "date", "type": "str", "example_values": null },\n  { "name": "type", "type": "str", "example_values": ["emopened", "emclickedlink", "emdelivery"] },\n  { "name": "description", "type": "str", "example_values": null },\n  { "name": "crm_id", "type": "int", "example_values": null },\n  { "name": "email", "type": "str", "example_values": null },\n  { "name": "ip_address", "type": "str", "example_values": null },\n  { "name": "user_agent", "type": "str", "example_values": null },\n  { "name": "bot", "type": "bool", "example_values": [false, true] },\n  { "name": "mailshot_id", "type": "int", "example_values": null },\n  { "name": "condition_id", "type": "int", "example_values": null },\n  { "name": "condition_title", "type": "str", "example_values": [\n    "Example 1", "Form Thank you - Kulea demo1", "Test2", "test run",\n    "Andrew Nicholson", "ccc", "eee", "Euro stagger

In [177]:
response = get_analysis(prompt, model)
response

"WITH event_data AS (SELECT email, type, LAG(type) OVER (PARTITION BY email ORDER BY date) AS previous_event_type FROM events) SELECT email AS user_email, type AS current_event_type, previous_event_type AS previous_event FROM event_data WHERE previous_event_type IS NOT NULL AND previous_event_type <> 'None' AND current_event_type = 'emopened'"

In [178]:
run_and_print_query(conn, response)

user_email | current_event_type | previous_event
---------------------------------------------
2116test@kulea.ma | emopened | emdelivery
28test@kulea.ma | emopened | emdelivery
3111test@kulea.ma | emopened | emdelivery
3237test@kulea.ma | emopened | emdelivery
7676test@kulea.ma | emopened | emdelivery
7676test@kulea.ma | emopened | emclickedlink
andrew@kulea.ma | emopened | emdelivery
andrew@kulea.ma | emopened | emdelivery
andrew@kulea.ma | emopened | emclickedlink
constantin@kulea.ma | emopened | emdelivery
constantin@kulea.ma | emopened | emdelivery
constantin@kulea.ma | emopened | emopened
constantin@kulea.ma | emopened | emdelivery
constantin@kulea.ma | emopened | emclickedlink
steve.bircher@btinternet.com | emopened | emdelivery
steve.bircher@btinternet.com | emopened | emopened
steve@kulea.ma | emopened | emdelivery
steve@kulea.ma | emopened | emopened
steve@kulea.ma | emopened | emclickedlink
steve@kulea.ma | emopened | emopened


In [111]:
prompt1 = f'''Based on the data set {json_data} and schema {schema}, create examples of natural language 
queries that can be used to generate SQL queries in order to test a text-2-SQL approach. Provides both 
simple query example based on one column and more complex queries based on multiples columns, including 
examples in which LAG and WITH SQL clauses are required.
'''
prompt1

'Based on the data set [{\'event_guid\': \'e5a86505-3727-4bbe-b357-a8c8ab231111\', \'date\': \'20250425-224145\', \'type\': \'emopened\', \'description\': \'4294967869 isEmailSent: true\', \'crm_id\': 4295002215, \'email\': \'andrew@kulea.ma\', \'ip_address\': \'103.107.197.125\', \'user_agent\': \'Mozilla/5.0\', \'bot\': False, \'mailshot_id\': 4294967869, \'condition_id\': 4294967860, \'condition_title\': \'Example 1\', \'condition_subject\': "Please don\'t open yet", \'condition_from\': \'andy@kulea.ma\', \'template_id\': None, \'template_title\': \'Test 1\', \'template_link\': \'Test-1\'}, {\'event_guid\': \'98c55d7e-5da8-45e7-9634-3127015913a3\', \'date\': \'20250424-214709\', \'type\': \'emopened\', \'description\': \'4294967869 isEmailSent: true\', \'crm_id\': 4295002273, \'email\': \'steve@kulea.ma\', \'ip_address\': \'172.224.227.3\', \'user_agent\': \'Mozilla/5.0\', \'bot\': False, \'mailshot_id\': 4294967869, \'condition_id\': 4294967860, \'condition_title\': \'Example 1\', 

In [112]:
response = get_analysis(prompt1, model)
display(to_markdown(response))

> Here are several examples of natural language queries that can be used to generate SQL queries based on the provided dataset and schema. The examples range from simple queries focusing on a single column to more complex queries involving multiple columns, including the use of `LAG` and `WITH` clauses.
> 
> ### Simple Queries (Single Column)
> 
> 1. **Query**: "Get all event GUIDs."
>    - **SQL**: `SELECT event_guid FROM events;`
> 
> 2. **Query**: "List all email addresses."
>    - **SQL**: `SELECT email FROM events;`
> 
> 3. **Query**: "Show all unique condition titles."
>    - **SQL**: `SELECT DISTINCT condition_title FROM events;`
> 
> 4. **Query**: "What are the types of events recorded?"
>    - **SQL**: `SELECT DISTINCT type FROM events;`
> 
> ### Intermediate Queries (Multiple Columns)
> 
> 5. **Query**: "Find all events of type 'emopened' with their corresponding email addresses."
>    - **SQL**: `SELECT event_guid, email FROM events WHERE type = 'emopened';`
> 
> 6. **Query**: "Get the CRM IDs and their corresponding condition titles for all events."
>    - **SQL**: `SELECT crm_id, condition_title FROM events;`
> 
> 7. **Query**: "List all events with the email 'andrew@kulea.ma' and their corresponding dates."
>    - **SQL**: `SELECT date, type FROM events WHERE email = 'andrew@kulea.ma';`
> 
> 8. **Query**: "Show the event GUIDs and descriptions for events that were opened by bots."
>    - **SQL**: `SELECT event_guid, description FROM events WHERE bot = TRUE;`
> 
> ### Complex Queries (Using LAG and WITH Clauses)
> 
> 9. **Query**: "Get the last event type for each email address."
>    - **SQL**:
>    ```sql
>    WITH RankedEvents AS (
>        SELECT email, type, 
>               LAG(type) OVER (PARTITION BY email ORDER BY date) AS previous_type
>        FROM events
>    )
>    SELECT email, type, previous_type FROM RankedEvents;
>    ```
> 
> 10. **Query**: "Find the first and last event dates for each email address."
>     - **SQL**:
>     ```sql
>     WITH DateRanges AS (
>         SELECT email, 
>                MIN(date) AS first_event_date, 
>                MAX(date) AS last_event_date
>         FROM events
>         GROUP BY email
>     )
>     SELECT * FROM DateRanges;
>     ```
> 
> 11. **Query**: "List all emails that opened an event and the previous event type they interacted with."
>     - **SQL**:
>     ```sql
>     WITH RankedEvents AS (
>         SELECT email, type, 
>                LAG(type) OVER (PARTITION BY email ORDER BY date) AS previous_type
>         FROM events
>     )
>     SELECT email, type, previous_type 
>     FROM RankedEvents 
>     WHERE type = 'emopened';
>     ```
> 
> 12. **Query**: "Get the count of events for each condition title and the last event date for each."
>     - **SQL**:
>     ```sql
>     WITH EventCounts AS (
>         SELECT condition_title, COUNT(*) AS event_count, 
>                MAX(date) AS last_event_date
>         FROM events
>         GROUP BY condition_title
>     )
>     SELECT * FROM EventCounts;
>     ```
> 
> These examples can be used to test a text-to-SQL approach by converting the natural language queries into SQL queries that can be executed against the provided dataset.